In [ ]:
# Importing Dependencies.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
##########################
import pandas as pd
import numpy as np
#########################
from scipy import stats
#########################
import cufflinks as cf
cf.go_offline()
#########################
import datetime as dt
from datetime import timedelta
##########################
import matplotlib.pyplot as plt
import pyfolio as pf
import empyrical as ep

In [ ]:
from tvDatafeed import TvDatafeed, Interval
##########################################
username = ''
password = ''
##########################################
tv = TvDatafeed(username, password)
# NIFTY INDEX.
Nifty_index_data = tv.get_hist(symbol='NIFTY',
                                    exchange='NSE',
                                    interval=Interval.in_3_minute,
                                    n_bars=20000)
Nifty_index_data = df = Nifty_index_data[['symbol','open','high','low','close','volume']]
df = df.drop(columns='volume',axis=1)
#df = df.between_time('9:15', '15:00')

In [ ]:
# Define the Effective Potential function
def effective_potential(high, low, close, open):
    return (abs(high - close) + (high - low) / 4 + abs(close - open) / 3)**2

# Define the backtest function
def backtest():
    # Retrieve historical stock data
    stock_data = df
    
    # Calculate the Effective Potential
    stock_data["Effective Potential"] = effective_potential(stock_data["high"], stock_data["low"], stock_data["close"], stock_data["open"])
    
    # Define the trading strategy
    stock_data["Signal"] = np.nan
    stock_data.loc[(stock_data["Effective Potential"] > stock_data["Effective Potential"].rolling(15).mean()) , "Signal"] = 1
    stock_data.loc[(stock_data["Effective Potential"] < stock_data["Effective Potential"].rolling(15).mean()) , "Signal"] = -1

    #stock_data['Signal'] = stock_data['Signal'].where(stock_data['Signal'].diff().fillna(0).ne(0), np.nan)
    stock_data['Signal'] = stock_data['Signal'].replace(to_replace=0,method='ffill')

    # Calculate the returns
    stock_data["Returns"] = stock_data["close"].pct_change() * stock_data["Signal"].shift(1)

    # Calculate the cumulative returns
    stock_data["Cumulative Returns"] = (stock_data["Returns"]).cumsum()
    
    # Print the final cumulative returns
    print(f"Final cumulative returns for {df['symbol'][-1]}: {stock_data['Cumulative Returns'][-1]}")
    
    # Plot the cumulative returns
    stock_data["Cumulative Returns"].iplot()#(figsize=(10, 6))
    
backtest()


In [ ]:
def metrics():
# Importing Required Dependencies.
    import numpy as np
    import pandas as pd
    import empyrical as ep
    import pyfolio as pf
# This code calculate the gains in points based on a signal column 
    j = df.Signal  # forward-fill missing values in df.position
    e = df.close

    result = []
    for i in range(len(j)):
        if i == len(j)-1 or pd.isna(j[i]) or pd.isna(j[i+1]):
            result.append(0)  # if there's a missing value or last row, append 0
        elif j[i] == -1:
            result.append(e[i] - e[i+1])
        elif j[i] == 1:
            result.append(e[i+1] - e[i])
        else:
            result.append(0)

    total = sum(result)

# Calculate the annualised Sharpe ratio
    sharpe = np.sqrt(252) * \
    df.Returns.mean() / df.Returns.std()
# Calculating Max Drawdown.
    max_dd= ep.max_drawdown(df.Returns)
# Calculating Stablity of Time Series of Returns.
    stablity = ep.stability_of_timeseries(df.Returns)
# Calculating Sortino Ratio
    Sortino = ep.sortino_ratio(df.Returns)
# Calculating Calmar ratio
    calmar_ratio = ep.calmar_ratio(df.Returns)
# Calculating Omega ratio
    omega_ratio = ep.omega_ratio(df.Returns, risk_free=0)
# Print start and end date of returns series
    print(f'Start: \n{df.index[0]} \nEnd: \n{df.index[-1]}')
    print('------------------------')
# Print various performance metrics
    print(f'Sortino Ratio: {(Sortino):.2f}') 
    print(f'Calmar Ratio: {calmar_ratio:.2f}')
    print(f'Omega Ratio: {omega_ratio:.2f}')
    print('Tail Ratio:',ep.tail_ratio(df['Returns']).round(2))
    print('------------------------------')
    print("The Annualised Sharpe Ratio is %.2f" % sharpe)
    print('------------------------------')
    print(f'Total Signal Counts:\n{df.Signal.value_counts()}')
    print('------------------------')
    print(f'Strategy Return: {((df.Returns.cumsum()[-1]*100)) :.2f} %') 
    print(f'Gains in Points: {total:.2f}')
    print(f'The Max Drawdown: {(max_dd*100):.2f} %') #times 100
    print(f'Stability of Time Series: {stablity:.2f}')
# Plot worst drawdown periods and monthly returns heatmap
    pf.show_worst_drawdown_periods(df['Returns'])
    pf.plot_monthly_returns_heatmap(df['Returns'])

metrics()

df['Signal'] = df['Signal'].where(df['Signal'].diff().fillna(0).ne(0), np.nan)

df['Signal'].value_counts()

In [ ]:
j = df.Signal  # forward-fill missing values in df.position
e = df.close

result = []
for i in range(len(j)):
    if i == len(j)-1 or pd.isna(j[i]) or pd.isna(j[i+1]):
        result.append(0)  # if there's a missing value or last row, append 0
    elif j[i] == -1:
        result.append(e[i] - e[i+1])
    elif j[i] == 1:
        result.append(e[i+1] - e[i])
    else:
        result.append(0)

total = sum(result)
print(f'Gains in Points: {total:.2f}')

In [ ]:
pf.create_full_tear_sheet(df['Returns'])

In [ ]:
df.head()
df.tail()